In [89]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [90]:
df = pd.read_csv('../data/player_stats_trends.csv')
tournament_stats = pd.read_csv('../data/tournament_stats_new.csv')
goals = pd.read_csv('../data/goals.csv')
awards = pd.read_csv('../data/awards.csv')

In [91]:
awards['Year'] = awards['Season'].apply(lambda x: float(str(x)[-2:]))
awards = awards.drop(columns=['Season', 'Club/Country'])
awards = awards.groupby('Award').filter(lambda x: len(x) >= 5)
awards = awards[~awards['Award'].isin(["Winner Ballon d'Or", 'Footballer of the Year', 'UEFA Best Player in Europe', "The Best FIFA Men's Player", 'Player of the Year', 'Top goal scorer', 'African Footballer of the Year'])]


awards = pd.get_dummies(awards, columns=['Award'], prefix='', prefix_sep='')

for col in awards.columns:
    if col not in ['Player', 'Year']:
        awards[col] = awards[col].astype(int)
awards = awards.groupby(['Player', 'Year']).sum().reset_index()


In [107]:
def goal_fixer(goals):
    try:
        return int(goals)
    except:
        return 0
def run_model(X, y):
    glm = sm.GLM(endog=y, exog= X, family=sm.families.Gaussian())
    X = sm.add_constant(X)
    glm_results = glm.fit()
    return glm_results
def remove_p(X, glm_results, p):
    to_remove = []
    for i in glm_results.pvalues.index:
        if glm_results.pvalues[i] > p:
        # print(i)
            to_remove.append(i)
        elif np.isnan(glm_results.pvalues[i]):
        # print(i)
            to_remove.append(i)
    X = X.drop(columns=to_remove)
    return X

In [93]:
tournament_stats['Year'] = tournament_stats['Season'].apply(lambda x: float(x[-2:]))
tournament_stats = tournament_stats.drop('Season', axis=1)
tournament_stats = tournament_stats.query('Year >= 8 & Year <= 23')
tournament_stats['Goals'] = tournament_stats['Goals'].apply(lambda x: goal_fixer(x))
tournament_stats['Minutes played'] = tournament_stats['Minutes played'].apply(lambda x: goal_fixer(x))

comp_goals = tournament_stats.pivot_table(index=['Year', 'Player'], columns='Competition', aggfunc='sum')['Goals'].replace(np.nan, 0).reset_index()
comp_goals = comp_goals.rename(columns=lambda x: x + '_Goals')
comp_goals.rename(columns={'Year_Goals': 'Year', 'Player_Goals' : 'Player'}, inplace=True)


comp_mins = tournament_stats.pivot_table(index=['Year', 'Player'], columns='Competition', aggfunc='sum')['Minutes played'].replace(np.nan, 0).reset_index()
comp_mins = comp_mins.rename(columns=lambda x: x + '_Minutes')
comp_mins.rename(columns={'Year_Minutes': 'Year', 'Player_Minutes' : 'Player'}, inplace=True)


In [94]:
df = df.drop(columns=['P1', 'P2', 'P3', 'P4', 'P5', 'Votes', 'RankPts', 'Percent', 'Voted', 'Season'])

In [95]:
for i in df.columns:
    if i  not in ['Player', 'Nationality', 'Club']:
        df[i] = df[i].astype(float)

In [96]:
# df = df.merge(comp_goals, on=['Year', 'Player'], how='left')
df = df.merge(awards, on=['Year', 'Player'], how='left')
df = df.merge(comp_mins, on=['Year', 'Player'], how='left')


df.fillna(0, inplace=True)

In [97]:
df.set_index(['Player', 'Year'], inplace=True)
X, y = df.drop(columns=['Share', 'Nationality', 'Club', 'Rank', 'Points', 'Month']), df['Share']
glm = sm.GLM(endog=y, exog= X, family=sm.families.Gaussian())
X = sm.add_constant(X)
glm_results = glm.fit()

In [99]:

remove_p()


In [108]:
run_model(X, y).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                  Share   No. Observations:                  365
Model:                            GLM   Df Residuals:                      350
Model Family:                Gaussian   Df Model:                           14
Link Function:               Identity   Scale:                       0.0036119
Method:                          IRLS   Log-Likelihood:                 516.23
Date:                Sat, 30 Dec 2023   Deviance:                       1.2628
Time:                        13:26:22   Pearson chi2:                     1.26
No. Iterations:                     3   Pseudo R-squ. (CS):             0.6018
Covariance Type:            nonrobust                                         
======================================================================================================
                                         coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
const                                 -0.0318      0.006     -5.157      0.000      -0.044      -0.020
Goals                                  0.0116      0.001     11.943      0.000       0.010       0.013
Assists                                0.0050      0.002      2.968      0.003       0.002       0.008
Goals conceded                         0.0047      0.001      3.931      0.000       0.002       0.007
UEFA-Cup_Qualifikation_appearances -1.186e-05   5.13e-06     -2.311      0.021   -2.19e-05    -1.8e-06
UI_Cup_appearances                 -1.186e-05   5.13e-06     -2.311      0.021   -2.19e-05    -1.8e-06
Brazilian cup winner                1.278e-17    5.2e-18      2.455      0.014    2.58e-18     2.3e-17
Champions League winner                0.0395      0.008      4.739      0.000       0.023       0.056
Dutch champion                        -0.0620      0.043     -1.446      0.148      -0.146       0.022
European champion                      0.0424      0.016      2.616      0.009       0.011       0.074
French Super Cup winner               -0.0359      0.020     -1.839      0.066      -0.074       0.002
German cup winner                     -0.0249      0.012     -1.994      0.046      -0.049      -0.000
Olympic medalist                       0.1148      0.035      3.260      0.001       0.046       0.184
Portuguese league cup winner       -1.164e-17   1.64e-17     -0.711      0.477   -4.37e-17    2.05e-17
Spanish Super Cup winner               0.0277      0.011      2.621      0.009       0.007       0.048
TM-Player of the season                0.0506      0.014      3.535      0.000       0.023       0.079
World Cup winner                       0.0446      0.016      2.762      0.006       0.013       0.076
Leagues Cup_Minutes                    0.0005   7.85e-05      6.179      0.000       0.000       0.001
UEFA-Cup Qualifikation_Minutes        -0.0009      0.000     -2.311      0.021      -0.002      -0.000
UI Cup_Minutes                        -0.0011      0.000     -2.311      0.021      -0.002      -0.000
======================================================================================================
"""

In [102]:
to_remove = []
for i in glm_results.pvalues.index:
    if glm_results.pvalues[i] > 0.05:
        # print(i)
        to_remove.append(i)
    elif np.isnan(glm_results.pvalues[i]):
        # print(i)
        to_remove.append(i)
X = X.drop(columns=to_remove)

In [103]:
glm = sm.GLM(endog=y, exog= X, family=sm.families.Gaussian())
X = sm.add_constant(X)
glm_results = glm.fit()
glm_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                  Share   No. Observations:                  365
Model:                            GLM   Df Residuals:                      350
Model Family:                Gaussian   Df Model:                           14
Link Function:               Identity   Scale:                       0.0036119
Method:                          IRLS   Log-Likelihood:                 516.23
Date:                Sat, 30 Dec 2023   Deviance:                       1.2628
Time:                        13:23:21   Pearson chi2:                     1.26
No. Iterations:                     3   Pseudo R-squ. (CS):             0.6018
Covariance Type:            nonrobust                                         
======================================================================================================
                                         coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
const                                 -0.0318      0.006     -5.157      0.000      -0.044      -0.020
Goals                                  0.0116      0.001     11.943      0.000       0.010       0.013
Assists                                0.0050      0.002      2.968      0.003       0.002       0.008
Goals conceded                         0.0047      0.001      3.931      0.000       0.002       0.007
UEFA-Cup_Qualifikation_appearances -1.186e-05   5.13e-06     -2.311      0.021   -2.19e-05    -1.8e-06
UI_Cup_appearances                 -1.186e-05   5.13e-06     -2.311      0.021   -2.19e-05    -1.8e-06
Brazilian cup winner                1.278e-17    5.2e-18      2.455      0.014    2.58e-18     2.3e-17
Champions League winner                0.0395      0.008      4.739      0.000       0.023       0.056
Dutch champion                        -0.0620      0.043     -1.446      0.148      -0.146       0.022
European champion                      0.0424      0.016      2.616      0.009       0.011       0.074
French Super Cup winner               -0.0359      0.020     -1.839      0.066      -0.074       0.002
German cup winner                     -0.0249      0.012     -1.994      0.046      -0.049      -0.000
Olympic medalist                       0.1148      0.035      3.260      0.001       0.046       0.184
Portuguese league cup winner       -1.164e-17   1.64e-17     -0.711      0.477   -4.37e-17    2.05e-17
Spanish Super Cup winner               0.0277      0.011      2.621      0.009       0.007       0.048
TM-Player of the season                0.0506      0.014      3.535      0.000       0.023       0.079
World Cup winner                       0.0446      0.016      2.762      0.006       0.013       0.076
Leagues Cup_Minutes                    0.0005   7.85e-05      6.179      0.000       0.000       0.001
UEFA-Cup Qualifikation_Minutes        -0.0009      0.000     -2.311      0.021      -0.002      -0.000
UI Cup_Minutes                        -0.0011      0.000     -2.311      0.021      -0.002      -0.000
======================================================================================================
"""

In [104]:
def predict_season(num):
    # print(X.iloc[num])
    pred_score = glm_results.predict(X.iloc[num]).values[0]
    print('Predicted Score:', pred_score, 'Actual Score: ', y.iloc[num])
predict_season(300)

Predicted Score: 0.015551125464934638 Actual Score:  0.0004


# Only using trends data

In [105]:
# X, y = df['Trend'], df['Share']
# glm = sm.GLM(endog=y, exog= X, family=sm.families.Gaussian())
# X = sm.add_constant(X)
# glm_results = glm.fit()

In [106]:
# glm_results.summary()